# Code to read COMBLE LES/SCM DEPHY forcing file and write input files for DHARMA LES (ASCII) and ModelE3 ESM SCM (NetCDF) examples
### Contributed by Ann Fridlind from NASA/GISS on 1/24/23

### Import libraries

In [5]:
import xarray as xr
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings("ignore")

### Read forcing files

In [2]:
dephy_filename = '../forcing/COMBLE_INTERCOMPARISON_NUDGING_FORCING.nc'
dephy = xr.open_dataset(dephy_filename)
dephy = dephy.squeeze()
dephy

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


<xarray.Dataset>
Dimensions:        (time: 29, lev: 136, zw_grid: 160)
Coordinates:
    t0             datetime64[ns] 2020-03-12T14:00:00
    lat            float32 77.6
    lon            float32 8.6
  * time           (time) datetime64[ns] 2020-03-12T14:00:00 ... 2020-03-13T1...
  * lev            (lev) float64 18.17 37.83 59.35 ... 6.851e+04 7.117e+04
  * zw_grid        (zw_grid) float64 0.0 20.0 45.0 ... 6.89e+03 6.95e+03 7e+03
Data variables: (12/13)
    pressure       (lev) float64 ...
    u              (lev) float64 ...
    v              (lev) float64 ...
    temp           (lev) float64 ...
    theta          (lev) float64 ...
    qv             (lev) float64 ...
    ...             ...
    thetas         float64 ...
    u_nudging      (time, lev) float64 ...
    v_nudging      (time, lev) float64 ...
    theta_nudging  (time, lev) float64 ...
    qv_nudging     (time, lev) float64 ...
    w_nudging      (time, lev) float64 ...
Attributes: (12/18)
    title:           Forcing and initial conditions for 13 March 2020 COMBLE ...
    reference:       https://arm-development.github.io/comble-mip/
    authors:         Timothy W. Juliano (NCAR/RAL, tjuliano@ucar.edu); Floria...
    version:         Created on 2023-02-01
    format_version:  DEPHY SCM format version 1
    script:          create_comble_nudging_forcing_era5ml.ipynb
    ...              ...
    dx:              150 m
    dy:              150 m
    dz:              see zw_grid variable
    nx:              648
    ny:              648
    nz:              160

In [ ]:
dephys_filename = 'COMBLE_INTERCOMPARISON_SFC_FORCING_ERA5ML_V1.6.nc'
dephys = xr.open_dataset(dephys_filename)
dephys

In [ ]:
# echo hourly values to enter into DHARMA input parameter file manually
dephys_h = dephys.resample(time="1H").nearest()
dephys_h.ts.round(1)

In [ ]:
plt.ylim([240,280])
plt.plot(dephys.time.values,dephys.ts.values)
plt.plot(dephys_h.time.values,dephys_h.ts.values)

In [ ]:
# test a more gradua increase in surface temperature
ts_new = dephys_h.ts.values.round(1)
ts_new

In [ ]:
ts_new[9] # last value at 247.

In [ ]:
ts_new[19] # roughly halfway through

In [ ]:
t_int = np.arange(9,13)
t_int

In [ ]:
np.interp(t_int,[9,13],[247.,275.2])

In [ ]:
ts_new[9:13] = np.interp(t_int,[9,13],[247.,275.2])
ts_new.round(1)

In [ ]:
plt.ylim([240,280])
plt.plot(dephys.time.values,dephys.ts.values)
plt.plot(dephys_h.time.values,dephys_h.ts.values)
plt.plot(dephys_h.time.values,ts_new)

### Write input sounding file for DHARMA LES in ASCII format

In [3]:
# use a list to accumulate output ASCII lines
lines_sounding = []

# number of lines to be read
n_z = dephy.dims['lev']
str_firstline = str(n_z+1) # number of heights after adding surface point below
lines_sounding.append(str_firstline)

# arrays to be reported in columns
z_arr = dephy.coords['lev'].values
th_arr = dephy['theta'].values.squeeze() # K
qt_arr = dephy['qv'].values.squeeze()*1000. # g/kg
u_arr = dephy['u'].values.squeeze() # m/s
v_arr = dephy['v'].values.squeeze() # m/s

# add a surface point (required by DHARMA code)
z0_arr = np.array([0.,th_arr[0],qt_arr[0],u_arr[0],v_arr[0]]) # well-mixed assumption
str_z0 = np.array2string(z0_arr,formatter={'float_kind':lambda z0_arr:"%12.4f" % z0_arr})[1:-1]
lines_sounding.append(str_z0)                        

for kk in range(n_z):
    vars_arr = np.array([z_arr[kk],th_arr[kk],qt_arr[kk],u_arr[kk],v_arr[kk]])
    str_vars = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%12.4f" % vars_arr})[1:-1]
    lines_sounding.append(str_vars)
    
str_headline = str('      z(m)        th(K)     qt(g/kg)       u(m/s)       v(m/s)')
lines_sounding.append(str_headline)

# write list contents to ASCII file
filename_sounding_LES = 'DHARMA_LES_ASCII_comble.input_sounding'
with open(filename_sounding_LES,mode='wt',encoding='utf-8') as sounding_file:
    sounding_file.write('\n'.join(lines_sounding))

### Write forcing file for DHARMA LES in ASCII format

In [6]:
# create thermodynamic forcing file contents as list
lines_forcing = []

# limit to bottom 8 km (e.g., subsidence invalid far aloft)
n_zf = np.max(np.where(dephy.lev<8000.))

n_t = dephy.dims['time']
str_firstline = str(n_zf+1)+' '+str(n_t) # extra line for z=0
lines_forcing.append(str_firstline) # number of heights, times

for tt in range(n_t):
    u_arr = dephy['u_nudging'][tt,:].values # m/s
    v_arr = dephy['v_nudging'][tt,:].values # m/s
    # w_arr = np.zeros(n_zf) # dummy values for subsidence, th and qv forcing (not used)
    om_arr = dephy['w_nudging'][tt,:].values # Pa/s
    p_arr = dephy['pressure'].values # Pa
    t_arr = dephy['temp'].values # K
    qv_arr = dephy['qv'].values # kg/kg
    rho_arr = p_arr/(t_arr*287.*(1.+0.608*qv_arr)) # kg/m3
    w_arr = -om_arr/(9.81*rho_arr.squeeze())*100. # cm/s
    th_arr = -np.ones(n_zf)
    qv_arr = -np.ones(n_zf) # not used for nudging
    
    z0_arr = np.array([0.,0.,-1.,-1.,u_arr[0],v_arr[0]])
    str_z0 = np.array2string(z0_arr,formatter={'float_kind':lambda z0_arr:"%10.2f" % z0_arr})[1:-1]
    lines_forcing.append(str_z0)
    
    for kk in range(n_zf):
        vars_arr = np.array([z_arr[kk],w_arr[kk],th_arr[kk],qv_arr[kk],u_arr[kk],v_arr[kk]])
        str_vars = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%10.2f" % vars_arr})[1:-1]
        lines_forcing.append(str_vars)
        
    lines_forcing.append(str(tt))
    
str_endlines1 = str('      z(m)    w(cm/s)      th(K)  qv(kg/kg)    u(m/s)    v(m/s)')
lines_forcing.append(str_endlines1)

str_endlines2 = str('time(h)')
lines_forcing.append(str_endlines2)

# write list contents to ASCII file
filename_forcing_LES = 'DHARMA_LES_ASCII_comble.input_forcing'
with open(filename_forcing_LES,mode='wt',encoding='utf-8') as forcing_file:
    forcing_file.write('\n'.join(lines_forcing))

In [ ]:
plt.ylim([0,7])
plt.plot(w_arr.squeeze(),dephy['lev'].values*1e-3)
plt.plot(om_arr.squeeze(),dephy['lev'].values*1e-3)

In [ ]:
Cval = -20.
inp = 0.25*(0.44e-6*np.exp(-0.57*Cval) - 0.00228*Cval**2 - 0.113*Cval - 1.22213)
print(Cval,inp)

### Write vertical grid file for DHARMA LES in ASCII format

In [ ]:
# create vertical grid file contents as list
lines_zw = []

n_zw = dephy.dims['zw_grid'] # number of heights
lines_zw.append(str(n_zw))

for kk in range(n_zw):
    lines_zw.append(str(dephy['zw_grid'][kk].values))

# write vertical grid to ASCII file
filename_zw_LES = 'DHARMA_LES_ASCII_comble.input_z'
with open(filename_zw_LES,mode='wt',encoding='utf-8') as zw_file:
    zw_file.write('\n'.join(lines_zw))

### Write initialization file for ModelE3 SCM in NetCDF format

In [ ]:
# open file with all components at initial time only
filename_init_SCM = 'ModelE3_SCM_COMBLE_init.nc'
scm_init = Dataset('./' + filename_init_SCM,mode='w',format='NETCDF3_CLASSIC')

# create dimensions
lev_dim = scm_init.createDimension('lev', n_z) # level axis is pressure for SCM
time_dim = scm_init.createDimension('time', 1) # initial time only

# create variables
lev = scm_init.createVariable('lev', np.float64, ('lev',))
lev.long_name = 'air pressure'
lev.units = 'mb'
lev[:] = dephy['pressure'].values

Ps = scm_init.createVariable('Ps', np.float64, ('time',))
Ps.long_name = 'surface pressure'
Ps.units = 'mb'
# Ps[:] = dephy['ps'].values
Ps[:] = 99407.19

T = scm_init.createVariable('T', np.float64, ('lev',))
T.long_name = 'air temperature'
T.units = 'K'
T[:] = dephy['temp'].values

Q = scm_init.createVariable('Q', np.float64, ('lev',))
Q.long_name = 'water vapor mixing ratio'
Q.units = 'kg kg-1'
Q[:] = dephy['qv'].values

# create time arrays
year = scm_init.createVariable('year', np.int16, ('time',))
year[:] = pd.to_datetime(dephy['time'].values[0]).year
month = scm_init.createVariable('month', np.int16, ('time',))
month[:] = pd.to_datetime(dephy['time'].values[0]).month
day = scm_init.createVariable('day', np.int16, ('time',))
day[:] = pd.to_datetime(dephy['time'].values[0]).day
hour = scm_init.createVariable('hour', np.int16, ('time',))
hour[:] = pd.to_datetime(dephy['time'].values[0]).hour

# close and dump
scm_init.close()
!ncdump ModelE3_SCM_COMBLE_init.nc

### Write forcing file for ModelE3 SCM in NetCDF format

In [ ]:
# open forcing file with all components on same time dimension
filename_forcing_SCM = 'ModelE3_SCM_COMBLE_force.nc'
scm_force = Dataset('./' + filename_forcing_SCM,mode='w',format='NETCDF3_CLASSIC')

# create dimensions
lev_dim = scm_force.createDimension('lev', n_z) # level axis is pressure for SCM
time_dim = scm_force.createDimension('time', n_t)

# create variables
lev = scm_force.createVariable('lev', np.float64, ('lev'))
lev.long_name = 'air pressure'
lev.units = 'mb'
lev[:] = dephy['pressure'].values

ts = scm_force.createVariable('Tskin', np.float32, ('time'))
ts.long_name = 'surface temperature'
ts.units = 'K'
ts[:] = dephy['ts'].values

ug = scm_force.createVariable('Ug', np.float64, ('time','lev',))
ug.units = 'm s-1'
ug.long_name = 'geostrophic zonal wind'
ug[:] = dephy['ug'].values

vg = scm_force.createVariable('Vg', np.float64, ('time','lev',))
vg.units = 'm s-1'
vg.long_name = 'geostrophic meridional wind'
vg[:] = dephy['vg'].values

# create time arrays
year = scm_force.createVariable('year', np.int16, ('time',))
year[:] = pd.to_datetime(dephy['time'].values).year
month = scm_force.createVariable('month', np.int16, ('time',))
month[:] = pd.to_datetime(dephy['time'].values).month
day = scm_force.createVariable('day', np.int16, ('time',))
day[:] = pd.to_datetime(dephy['time'].values).day
hour = scm_force.createVariable('hour', np.int16, ('time',))
hour[:] = pd.to_datetime(dephy['time'].values).hour

# close
scm_force.close()
!ncdump ModelE3_SCM_COMBLE_force.nc